### Carrega Processos Baixados do Tribunal no Banco

1. Carrega o arquivo CSV que foi baixado no passo "Encontra Processos"
2. Para Cada linha no arquivo, inclui/atualiza o documento na base 




In [1]:
# Importando as Bibliotecas
import pandas as pd
import json
import pymongo

In [2]:
# Conexão com o Banco de Dados
user = ""
client = pymongo.MongoClient("mongodb+srv://" + user + "@cluster0.kqbxfp4.mongodb.net/?retryWrites=true&w=majority")
db = client['proc-jud']
col_processos = db['processo']

In [3]:
#Parâmetros de Carga
arquivo = "2022_09_22_tjsp_processos_mapfre.csv"


In [4]:
# Carrega o arquivo CSV em um Data Frame
df_processos = pd.read_csv('./dados/' + arquivo,sep=';',encoding='utf-8')
df_processos

,nro_processo,link_processo,nome_parte,classe_processo,assunto_processo,data_local_distribuicao,cod_empresa
0,1047365-05.2022.8.26.0053,/cpopg/show.do?processo.codigo=1H000MWP90000&p...,Mapfre Vera Cruz Seguradora S/A,Procedimento Comum Cível,Acidente de Trânsito,12/08/2022 - 4ª Vara de Fazenda Pública,MAPFRE
1,1044048-96.2022.8.26.0053,/cpopg/show.do?processo.codigo=1H000MSXX0000&p...,Mapfre Vera Cruz Seguradora S/A,Procedimento Comum Cível,Acidente de Trânsito,28/07/2022 - 11ª Vara de Fazenda Pública,MAPFRE
2,0045187-52.2012.8.26.0053,/cpopg/show.do?processo.codigo=1H0004KXV0000&p...,Mapfre Vera Cruz Seguradora S/A,Procedimento Comum Cível,Pagamento,25/09/2012 - Unidade de Processamento das Exec...,MAPFRE
3,1043418-40.2022.8.26.0053,/cpopg/show.do?processo.codigo=1H000MS750000&p...,Mapfre Vera Cruz Seguradora S/A,Procedimento Comum Cível,Acidente de Trânsito,26/07/2022 - 4ª Vara de Fazenda Pública,MAPFRE
4,1076017-25.2021.8.26.0002,/cpopg/show.do?processo.codigo=020028XQU0000&p...,Mapfre Vera Cruz Seguradora S/A,Procedimento Comum Cível,Seguro,20/12/2021 - 4ª Vara de Fazenda Pública,MAPFRE
...,...,...,...,...,...,...,...
5289,1003799-66.2017.8.26.0704,/cpopg/show.do?processo.codigo=JK0001O6B0000&p...,Mapfre Seguros Gerais S.A.,Procedimento Comum Cível,Seguro,17/06/2017 - 3ª Vara Cível,MAPFRE
5290,1003624-72.2017.8.26.0704,/cpopg/show.do?processo.codigo=JK0001NXJ0000&p...,Mapfre Seguros Gerais S.A.,Procedimento Comum Cível,Seguro,08/06/2017 - 3ª Vara Cível,MAPFRE
5291,1006994-93.2016.8.26.0704,/cpopg/show.do?processo.codigo=JK0001F1N0000&p...,Mapfre Seguros Gerais S.A.,Execução de Título Extrajudicial,Locação de Imóvel,25/10/2016 - 2ª Vara Cível,MAPFRE
5292,1008391-27.2015.8.26.0704,/cpopg/show.do?processo.codigo=JK00015GF0000&p...,Mapfre Seguros Gerais S.A.,Procedimento Comum Cível,Indenização por Dano Material,18/12/2015 - 3ª Vara Cível,MAPFRE


In [5]:
# Carrega os dados do processo na base, se já existe, atualiza. 
for index,proc in df_processos.iterrows(): 
    json_str = df_processos.loc[index].to_json()
    json_str = json_str.replace('[','')
    json_str = json_str.replace(']','')
    json_str = '{"processo":' + json_str + '}'
    col_processos.find_one_and_update({'processo.nro_processo':df_processos.loc[0]['nro_processo']},{'$set':json.loads(json_str)},upsert=True)    
    

    